# Letter Recognition

<img src="images/A.jpg"/>

One of the earliest applications of the predictive analytics methods we have studied so far in this class was to automatically recognize letters, which post office machines use to sort mail. In this problem, we will build a model that uses statistics of images of four letters in the Roman alphabet -- A, B, P, and R -- to predict which letter a particular image corresponds to.

Note that this is a multiclass classification problem. We have mostly focused on binary classification problems (e.g., predicting whether an individual voted or not, whether the Supreme Court will affirm or reverse a case, whether or not a person is at risk for a certain disease, etc.). In this problem, we have more than two classifications that are possible for each observation, like in the D2Hawkeye lecture. 

The file letters_ABPR.csv contains 3116 observations, each of which corresponds to a certain image of one of the four letters A, B, P and R. The images came from 20 different fonts, which were then randomly distorted to produce the final images; each such distorted image is represented as a collection of pixels, each of which is "on" or "off". For each such distorted image, we have available certain statistics of the image in terms of these pixels, as well as which of the four letters the image is. This data comes from the UCI Machine Learning Repository.

### The Data

This dataset contains the following 17 variables:

    letter = the letter that the image corresponds to (A, B, P or R)
    
    xbox = the horizontal position of where the smallest box covering the letter shape begins.
    
    ybox = the vertical position of where the smallest box covering the letter shape begins.
    
    width = the width of this smallest box.
    
    height = the height of this smallest box.
    
    onpix = the total number of "on" pixels in the character image
    
    xbar = the mean horizontal position of all of the "on" pixels
    
    ybar = the mean vertical position of all of the "on" pixels
    
    x2bar = the mean squared horizontal position of all of the "on" pixels in the image
    
    y2bar = the mean squared vertical position of all of the "on" pixels in the image
    
    xybar = the mean of the product of the horizontal and vertical position of all of the "on" pixels in the image
    
    x2ybar = the mean of the product of the squared horizontal position and the vertical position of all of the "on" pixels
    
    xy2bar = the mean of the product of the horizontal position and the squared vertical position of all of the "on" pixels
    
    xedge = the mean number of edges (the number of times an "off" pixel is followed by an "on" pixel, or the image boundary is hit) as the image is scanned from left to right, along the whole vertical length of the image
    
    xedgeycor = the mean of the product of the number of horizontal edges at each vertical position and the vertical position
    
    yedge = the mean number of edges as the images is scanned from top to bottom, along the whole horizontal length of the image
    
    yedgexcor = the mean of the product of the number of vertical edges at each horizontal position and the horizontal position

### Load and Exploration the Data

In [1]:
letters = read.csv("data/letters_ABPR.csv")
head(letters)

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgeycor,yedge,yedgexcor
,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
2,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
3,R,5,9,5,7,6,6,11,7,3,7,3,9,2,7,5,11
4,B,5,9,7,7,10,9,8,4,4,6,8,6,6,11,8,7
5,P,3,6,4,4,2,4,14,8,1,11,6,3,0,10,4,8
6,R,8,10,8,6,6,7,7,3,5,8,4,8,6,6,7,7


In [2]:
str(letters)

'data.frame':	3116 obs. of  17 variables:
 $ letter   : Factor w/ 4 levels "A","B","P","R": 2 1 4 2 3 4 4 1 3 3 ...
 $ xbox     : int  4 1 5 5 3 8 2 3 8 6 ...
 $ ybox     : int  2 1 9 9 6 10 6 7 14 10 ...
 $ width    : int  5 3 5 7 4 8 4 5 7 8 ...
 $ height   : int  4 2 7 7 4 6 4 5 8 8 ...
 $ onpix    : int  4 1 6 10 2 6 3 3 4 7 ...
 $ xbar     : int  8 8 6 9 4 7 6 12 5 8 ...
 $ ybar     : int  7 2 11 8 14 7 7 2 10 5 ...
 $ x2bar    : int  6 2 7 4 8 3 5 3 6 7 ...
 $ y2bar    : int  6 2 3 4 1 5 5 2 3 5 ...
 $ xybar    : int  7 8 7 6 11 8 6 10 12 7 ...
 $ x2ybar   : int  6 2 3 8 6 4 5 2 5 6 ...
 $ xy2bar   : int  6 8 9 6 3 8 7 9 4 6 ...
 $ xedge    : int  2 1 2 6 0 6 3 2 4 3 ...
 $ xedgeycor: int  8 6 7 11 10 6 7 6 10 9 ...
 $ yedge    : int  7 2 5 8 4 7 5 3 4 8 ...
 $ yedgexcor: int  10 7 11 7 8 7 8 8 8 9 ...


In [3]:
summary(letters)

 letter       xbox             ybox            width            height      
 A:789   Min.   : 0.000   Min.   : 0.000   Min.   : 1.000   Min.   : 0.000  
 B:766   1st Qu.: 3.000   1st Qu.: 5.000   1st Qu.: 4.000   1st Qu.: 4.000  
 P:803   Median : 4.000   Median : 7.000   Median : 5.000   Median : 6.000  
 R:758   Mean   : 3.915   Mean   : 7.051   Mean   : 5.186   Mean   : 5.276  
         3rd Qu.: 5.000   3rd Qu.: 9.000   3rd Qu.: 6.000   3rd Qu.: 7.000  
         Max.   :13.000   Max.   :15.000   Max.   :11.000   Max.   :12.000  
     onpix             xbar             ybar            x2bar       
 Min.   : 0.000   Min.   : 3.000   Min.   : 0.000   Min.   : 0.000  
 1st Qu.: 2.000   1st Qu.: 6.000   1st Qu.: 6.000   1st Qu.: 3.000  
 Median : 4.000   Median : 7.000   Median : 7.000   Median : 4.000  
 Mean   : 3.869   Mean   : 7.469   Mean   : 7.197   Mean   : 4.706  
 3rd Qu.: 5.000   3rd Qu.: 8.000   3rd Qu.: 9.000   3rd Qu.: 6.000  
 Max.   :12.000   Max.   :14.000   Max.   :15.0

### Problem 1.1 - Predicting B or not B
Let's warm up by attempting to predict just whether a letter is B or not. To begin, load the file letters_ABPR.csv into R, and call it letters. Then, create a new variable isB in the dataframe, which takes the value "TRUE" if the observation corresponds to the letter B, and "FALSE" if it does not. You can do this by typing the following command into your R console:

    letters$isB = as.factor(letters$letter == "B")

Now split the data set into a training and testing set, putting 50% of the data in the training set. Set the seed to 1000 before making the split. The first argument to sample.split should be the dependent variable "letters$isB". Remember that TRUE values from sample.split should go in the training set.

Before building models, let's consider a baseline method that always predicts the most frequent outcome, which is "not B". What is the accuracy of this baseline method on the test set?

In [4]:
# Convert to factor
letters$isB = as.factor(letters$letter == "B")

head(letters)

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgeycor,yedge,yedgexcor,isB
,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>
1,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10,TRUE
2,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7,FALSE
3,R,5,9,5,7,6,6,11,7,3,7,3,9,2,7,5,11,FALSE
4,B,5,9,7,7,10,9,8,4,4,6,8,6,6,11,8,7,TRUE
5,P,3,6,4,4,2,4,14,8,1,11,6,3,0,10,4,8,FALSE
6,R,8,10,8,6,6,7,7,3,5,8,4,8,6,6,7,7,FALSE


In [5]:
library(caTools)

In [6]:
# Split the data
set.seed(1000)

spl = sample.split(letters$isB, SplitRatio = 0.5)

train = subset(letters, spl == TRUE)
test = subset(letters, spl == FALSE)

In [7]:
# Tabulate if not B
isB = table(test$isB)
isB


FALSE  TRUE 
 1175   383 

In [8]:
# baseline method that always predicts is "not B"

accnotb = isB[1]/sum(isB)  #  Right when is False, Number = Falses/(True + Falses)
paste("Accuracy always not B: ", round(accnotb ,4))

[1] "Accuracy always not B:  0.7542"

### Problem 1.2 - Predicting B or not B
Now build a classification tree to predict whether a letter is a B or not, using the training set to build your model. Remember to remove the variable "letter" out of the model, as this is related to what we are trying to predict! To just remove one variable, you can either write out the other variables, or remember what we did in the Billboards problem in Week 3, and use the following notation:

    CARTb = rpart(isB ~ . - letter, data=train, method="class")

We are just using the default parameters in our CART model, so we don't need to add the minbucket or cp arguments at all. We also added the argument method="class" since this is a classification problem.

What is the accuracy of the CART model on the test set? (Use type="class" when making predictions on the test set.)

In [9]:
library(rpart)
library(rpart.plot)

In [10]:
# Cart Model
CARTb = rpart(isB ~ . - letter, data=train, method="class")

# Make predictions
predictions = predict(CARTb, newdata=test, type="class")

In [11]:
# Confusion matrix
cm = table(test$isB, predictions)
cm

       predictions
        FALSE TRUE
  FALSE  1118   57
  TRUE     43  340

In [12]:
# Accuracy CART Model
accur = sum(diag(cm))/sum(cm)  # (TN+TP)/(TN+FN+TP+FP)
paste("Accuracy CART Model: ", round(accur,digits=4))

[1] "Accuracy CART Model:  0.9358"

### Problem 1.3 - Predicting B or Not B
Now, build a random forest model to predict whether the letter is a B or not (the isB variable) using the training set. You should use all of the other variables as independent variables, except letter (since it helped us define what we are trying to predict!). Use the default settings for ntree and nodesize (don't include these arguments at all). Right before building the model, set the seed to 1000. (NOTE: You might get a slightly different answer on this problem, even if you set the random seed. This has to do with your operating system and the implementation of the random forest algorithm.)

What is the accuracy of the model on the test set?

In [13]:
library(randomForest)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.



In [14]:
# Random Forest model
set.seed(1000)

RFb = randomForest(isB ~ . - letter, data=train)

# Make predictions
predictions = predict(RFb, newdata=test)

In [15]:
# Confusion matrix
cmRF = table(test$isB, predictions)
cmRF

       predictions
        FALSE TRUE
  FALSE  1163   12
  TRUE      9  374

In [16]:
# Accuracy Random Forest
accurRF = sum(diag(cmRF))/sum(cmRF)  # (TN+TP)/(TN+FN+TP+FP)
paste("Accuracy Random Forest: ", round(accurRF,digits=4))

[1] "Accuracy Random Forest:  0.9865"

In lecture, we noted that random forests tends to improve on CART in terms of predictive accuracy. Sometimes, this improvement can be quite significant, as it is here.

### Problem 2.1 - Predicting the letters A, B, P, R
Let us now move on to the problem that we were originally interested in, which is to predict whether or not a letter is one of the four letters A, B, P or R.

As we saw in the D2Hawkeye lecture, building a multiclass classification CART model in R is no harder than building the models for binary classification problems. Fortunately, building a random forest model is just as easy.

The variable in our data frame which we will be trying to predict is "letter". Start by converting letter in the original data set (letters) to a factor by running the following command in R:

    letters$letter = as.factor( letters$letter )

Now, generate new training and testing sets of the letters data frame using letters$letter as the first input to the sample.split function. Before splitting, set your seed to 2000. Again put 50% of the data in the training set. (Why do we need to split the data again? Remember that sample.split balances the outcome variable in the training and testing sets. With a new outcome variable, we want to re-generate our split.)

In a multiclass classification problem, a simple baseline model is to predict the most frequent class of all of the options.

What is the baseline accuracy on the testing set?

In [17]:
# Convert to factor
letters$letter = as.factor( letters$letter ) 

head(letters)

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgeycor,yedge,yedgexcor,isB
,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>
1,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10,TRUE
2,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7,FALSE
3,R,5,9,5,7,6,6,11,7,3,7,3,9,2,7,5,11,FALSE
4,B,5,9,7,7,10,9,8,4,4,6,8,6,6,11,8,7,TRUE
5,P,3,6,4,4,2,4,14,8,1,11,6,3,0,10,4,8,FALSE
6,R,8,10,8,6,6,7,7,3,5,8,4,8,6,6,7,7,FALSE


In [18]:
# Split the data
set.seed(2000)

spl = sample.split(letters$letter, SplitRatio = 0.5)

train2 = subset(letters, spl == TRUE)
test2 = subset(letters, spl == FALSE)

In [19]:
# Tabulate letters
bl = table(test2$letter)
bl


  A   B   P   R 
395 383 401 379 

In [20]:
# Baseline accuracy on the testing set
accurts = max(bl)/sum(bl)  # predict the most frequent class of all of the options
paste("Accuracy BL testing set: ", round(accurts,digits=4))

[1] "Accuracy BL testing set:  0.2574"

### Problem 2.2 - Predicting the letters A, B, P, R
Now build a classification tree to predict "letter", using the training set to build your model. You should use all of the other variables as independent variables, except "isB", since it is related to what we are trying to predict! Just use the default parameters in your CART model. Add the argument method="class" since this is a classification problem. Even though we have multiple classes here, nothing changes in how we build the model from the binary case.

What is the test set accuracy of your CART model? Use the argument type="class" when making predictions.

(HINT: When you are computing the test set accuracy using the confusion matrix, you want to add everything on the main diagonal and divide by the total number of observations in the test set, which can be computed with nrow(test), where test is the name of your test set).

In [21]:
# CART Model
CARTletter = rpart(letter ~ . - isB, data=train2, method="class")

# Make predictions
predictLetter = predict(CARTletter, newdata=test2, type="class")

In [22]:
# Confusion Matrix
cmCM = table(test2$letter, predictLetter)
cmCM

   predictLetter
      A   B   P   R
  A 348   4   0  43
  B   8 318  12  45
  P   2  21 363  15
  R  10  24   5 340

In [23]:
# Compute accuracy
accurCM = sum(diag(cmCM))/sum(cmCM)
paste("Accuracy Cart Model2: ", round(accurCM,digits=4))

[1] "Accuracy Cart Model2:  0.8787"

### Problem 2.3 - Predicting the letters A, B, P, R
Now build a random forest model on the training data, using the same independent variables as in the previous problem -- again, don't forget to remove the isB variable. Just use the default parameter values for ntree and nodesize (you don't need to include these arguments at all). Set the seed to 1000 right before building your model. (Remember that you might get a slightly different result even if you set the random seed.)

What is the test set accuracy of your random forest model?

In [24]:
# Random Forest model
set.seed(1000)
RFletter = randomForest(letter ~ . - isB, data=train2)

# Make predictions
predictLetter = predict(RFletter, newdata=test2)

In [25]:
# Confusion matrix
cmFM = table(test2$letter, predictLetter)
cmFM

   predictLetter
      A   B   P   R
  A 391   0   3   1
  B   0 380   1   2
  P   0   6 394   1
  R   3  14   0 362

In [26]:
# Compute accuracy
accurFM = sum(diag(cmFM))/sum(cmFM)
paste("Accuracy Random Forest2: ", round(accurFM,digits=4))

[1] "Accuracy Random Forest2:  0.9801"

You should find this value rather striking, for several reasons. The first is that it is significantly higher than the value for CART, highlighting the gain in accuracy that is possible from using random forest models. The second is that while the accuracy of CART decreased significantly as we transitioned from the problem of predicting B/not B (a relatively simple problem) to the problem of predicting the four letters (certainly a harder problem), the accuracy of the random forest model decreased by a tiny amount.